## Loading the LLM

In [34]:
from dotenv import load_dotenv
from typing import TypedDict, List, Union

# Lang Graph Imports
from langgraph.graph import StateGraph, START, END

# Langchin Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # Or "gemini-1.5-pro", etc.
    temperature=0,  # For more deterministic responses
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Basic AI Agent 1

In [35]:
class AgentState(TypedDict):
    messages: List[HumanMessage]


def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(response.content)
    return state


graph = StateGraph(AgentState)
graph.add_node("process_node", process)
graph.add_edge(START, "process_node")
graph.add_edge("process_node", END)
agent = graph.compile()

user_input = input("Enter your message: ")
agent.invoke({"messages": [HumanMessage(content=user_input)]})

/Users/rifat/Documents/Coding/lang_graph_learning/.venv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:1568: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents: contents is not specified


### Agent 2 (Context)

In [ ]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]


def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(response.content)
    state["messages"].append(AIMessage(content=response.content))
    print(f"Current State Messages:{state['messages']}")
    return state


graph2 = StateGraph(AgentState)
graph2.add_node("process_node", process)
graph2.add_edge(START, "process_node")
graph2.add_edge("process_node", END)
agent = graph2.compile()
user_input = input("Enter your message: ")
conversation_history = []
while user_input.lower() != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    # Update conversation_history with the AI response
    conversation_history = result["messages"]

    # Remove oldest pair if more than 3 pairs (6 messages)
    if len(conversation_history) > 6:
        conversation_history = conversation_history[
            2:
        ]  # Remove first pair (2 messages)

    user_input = input("Enter your message: ")

Hi there! How can I help you today?
Current State Messages:[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]
There isn't a single "best" country in the world, as it depends entirely on what criteria you prioritize. What one person considers the best, another might not.

However, if you're looking for countries that frequently rank highly in various global indices for quality of life, happiness, economic stability, and social progress, some commonly cited examples include:

*   **Switzerland**
*   **Canada**
*   **Norway**
*   **Sweden**
*   **New Zealand**
*   **Finland**
*   **Denmark**
*   **Australia**
*   **Netherlands**
*   **Iceland**

Again, this is just a selection based on common rankings, and the "best" is subjective.
Current State Messages:[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I he

### Agent : ReAct (Reasoning and Acting Agent)

![image.png](attachment:image.png)

In [ ]:
from typing import Annotated, Sequence

# Message for providing instructions to LLM
from langchain_core.messages import SystemMessage

# passes data back to LLM after it calls a tool
from langchain_core.messages import ToolMessage

# The foundational class for all message types in LangGraph
from langchain_core.messages import BaseMessage

from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# ===== example =======

# Annotated - provides additional context witout affecting the type itself

email = Annotated[str, "This has to be valid email"]

email.__metadata__  ## output ('This has to be valid email',)

# Sequence: To automatically handle the state updates for sequences such as by adding new messages to a chat history
# Reducer: how to merge new data into existing state
# add_messaes is the reducer here. It appends new messages to the existing list of messages in the state.

# ====== Example END ======

('This has to be valid email',)

In [49]:
class AgentState(TypedDict):
    # preserve the state by appending. Do not override
    messages: Annotated[Sequence[BaseMessage], add_messages]


@tool
def add(a: int, b: int) -> int:
    """This tool adds two numbers together."""
    return a + b


@tool
def date_tool() -> str:
    """Returns the current date."""
    from datetime import datetime

    return datetime.now().strftime("%Y-%m-%d")


tools = [add, date_tool]

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # Stable model with good free tier support
    temperature=0,  # For more deterministic responses
    max_tokens=None,
    timeout=None,
    max_retries=2,
).bind_tools(tools)


def model_call(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(
        content="You are my AI assistant, please answer my query to the best of your ability."
    )
    response = llm.invoke([system_prompt] + state["messages"])
    print(f"Model Response: {response.content}")
    ## response will be appended by reducer function no need to append manually
    return {"messages": response}


def should_continue(state: AgentState) -> str:
    last_message = state["messages"][-1]
    if not last_message.tool_calls:
        return "end"
    return "continue"


def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


graph = StateGraph(AgentState)
graph.add_node("model_call", model_call)
tool_node = ToolNode(tools=tools)
graph.add_node("tool_node", tool_node)
graph.set_entry_point("model_call")

graph.add_conditional_edges(
    "model_call",
    should_continue,
    {
        "continue": "tool_node",
        "end": END,
    },
)
graph.add_edge("tool_node", "model_call")

app = graph.compile()

inputs = {"messages": [("user", "add 3 and 4")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

add 3 and 4


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 35.524419172s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelangu

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 33.343455708s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeT